<a href="https://colab.research.google.com/github/BullInBoots/streamlit-demo-2/blob/main/objRec_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%%writefile .gitignore
/node_modules
/sample_data

Overwriting .gitignore


In [ ]:
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 2.123s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
%%writefile -a app.py
import streamlit as st
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes

Writing app.py


In [ ]:
%%writefile -a app.py
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
categories = weights.meta["categories"]
img_preprocess = weights.transforms()

Appending to app.py


In [ ]:
%%writefile -a app.py
@st.cache_resource
def load_model():
  model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.8)
  model.eval()
  return model

model = load_model()

Appending to app.py


In [ ]:
%%writefile -a app.py
def make_prediction(img):
  img_processed = img_preprocess(img)
  prediction = model(img_processed.unsqueeze(0))
  prediction = prediction[0]
  prediction["labels"] = [categories[label] for label in prediction["labels"]]
  return prediction

def create_image_with_boxes(img, prediction):
  img_tensor = torch.tensor(img)
  img_with_boxes = draw_bounding_boxes(
      img_tensor,
      boxes=prediction["boxes"],
      labels=prediction["labels"],
      colors=["red" if label=="person" else "green" for label in prediction["labels"]],
      width=2
  )
  img_with_boxes_np = img_with_boxes.detach().numpy().transpose(1, 2, 0)
  return img_with_boxes_np

Appending to app.py


In [ ]:
%%writefile -a app.py
st.title("Object Detector :tea :coffee:")
upload = st.file_uploader(label="Upload Image Here:", type=["png", "jpg", "jpeg"])

if upload:
  img = Image.open(upload)

  prediction = make_prediction(img)
  img_with_box = create_image_with_boxes(
      np.array(img).transpose(2, 0, 1),
      prediction
  )

  fig = plt.figure(figsize=(12, 12))
  ax = fig.add_subplot(111)
  plt.imshow(img_with_box)
  plt.xticks([], [])
  plt.yticks([], [])
  ax.spines[["top", "bottom", "right", "left"]].set_visible(False)

  st.pyplot(fig, use_container_width=True)
  del prediction["boxes"]
  st.header("Prediction Probabilities")
  st.write(prediction)

Appending to app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.82.150.178
npx: installed 22 in 3.019s
your url is: https://fruity-planes-cheer.loca.lt
